In [1]:
import sklearn
from sklearn import cluster
import folium

import pandas as pd
import numpy as np
import random
from threading import Thread

import matplotlib.pyplot as plt
#import matplotlib.colors as mpl_colors

from IPython.display import clear_output, display, HTML
import sys  # This is necessary for printing updates within a code block, 
            #   via sys.stdout.flush()
import time # Use time.sleep(secs) to sleep a process if needed
%matplotlib inline

print("Current environment directory:" + sys.prefix)
print("System version: "+sys.version)

Current environment directory:/Users/emunsing/anaconda
System version: 2.7.10 |Anaconda 2.1.0 (x86_64)| (default, Oct 19 2015, 18:31:17) 
[GCC 4.2.1 (Apple Inc. build 5577)]


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days


In [8]:
# In the future, handle household data separately
householdData = pd.read_csv('Dummy_SolarData.csv')

# Utility-level parameters; should be fixed for the whole region
purchaseRate = householdData['purchaseRate']  # This will ultimately be set by 
feedInRate = householdData['feedInRate']

,hour,loadProfile,genProfile,purchaseRate,feedInRate
0,0,0.5,0,0.12,0.02
1,1,0.5,0,0.12,0.02
2,2,0.5,0,0.12,0.02
3,3,0.5,0,0.12,0.02
4,4,0.5,0,0.12,0.02


'1234 Anystreet'

In [30]:

# Load list of households and associated usage profiles.
# For each household
#  Calculate its bill
# Pass block of household data to mapping function

# Assume household data is stored as rows in a Pandas dataframe
# Assume we have separate dataframes of household Load and Generation profiles

householdData = {}

# Household-level data
loadProfile = householdData['loadProfile']
genProfile = householdData['genProfile']

baselineBill = sum(loadProfile * purchaseRate)
print("Baseline Bill: $%.2f" %(baselineBill))

solarBill = calculateDailyProfits(loadProfile, genProfile, purchaseRate, feedInRate, dT=1, battery=0 isNEM=False)
print("Bill with Solar: $%.2f" %(solarBill) )

solarSavings = baselineBill - solarBill
print("Savings: $%.2f" %(solarSavings))



Baseline Bill: $3.06
Bill with Solar: $1.65
Savings: $1.41


In [24]:
def calculateDailyProfits(loadProfile, genProfile, purchaseRate, feedInRate, dT=1, isNEM=False):
    # Assume that LoadProfile, solarGen, and utilityRate are nparrays with the same length.
    #   Initially assume that these are 1-hour blocks, 24 total
    # LoadProfile: Array of 24 hourly blocks; kW
    # genProfile: Array of 24 hourly blocks; kW
    # purchaseRate: Array of 24 hourly blocks for retail utility purchase. $/kWh
    # feedInRate: Array of 24 hourly blocks for payments of extra power. $/kWh
    # dT : time step for each block. dT=1 : 1-hour blocks.
    # isNEM : Boolean indicator of whether energy is metered in net for the billing period (TRUE) or instantaneously (FALSE)
    #         if TRUE, payment will only be calculated based on the first entry of purchaseRate and feedInRate
    
    netLoad = loadProfile - genProfile  # Positive values: Importing energy. Negative: Exporting energy
    
    if isNEM:
        # Need to calculate net energy consumption over the study period
        totalEnergy = sum(netLoad) * dT
        if totalEnergy >= 0:
            bill = totalEnergy * purchaseRate[0]
        else:
            bill = totalEnergy * feedInRate[0]
    else:
        expenses = sum( netLoad[netLoad >=0] * purchaseRate[netLoad >= 0])
        income = sum( netLoad[netLoad < 0] * feedInRate[netLoad < 0])
        bill = expenses + income
        
    return bill